In [17]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [18]:
file_path = 'Tracks_Final.csv'
ratings=pd.read_csv(file_path, sep = '\t', header=0, names = ["user_id","track_name","rating"] )
ratings = ratings.loc[:,["user_id","track_name","rating"]]
ratings

,user_id,track_name,rating
0,1,'84 Pontiac Dream,1.000000
1,1,(All I Wanted Was To Make You) Feel Good,0.214286
2,1,(D)Ecaying Beauty,0.071429
3,1,(Far From) Home,0.071429
4,1,(Get Off Your) High Horse Lady,0.071429
...,...,...,...
4407925,1000,Zero,0.178571
4407926,1000,Zgon,0.178571
4407927,1000,Zung Zung Funky Music,0.089286
4407928,1000,Züri,0.089286


In [19]:
ratings.rating.describe()

count    4.407930e+06
mean     2.748148e-01
std      4.036867e-01
min      2.360000e-03
25%      6.622500e-02
50%      1.388890e-01
75%      3.076920e-01
max      5.000000e+00
Name: rating, dtype: float64

In [20]:
reader = Reader( rating_scale = ( 0, 5.00 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ ["user_id","track_name","rating"] ], reader )

In [6]:
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)

In [4]:
#del ratings

In [6]:
train_set.n_ratings

717936

In [7]:
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)

In [8]:
predictions = algo.fit(train_set).test(test_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [89]:
predictions[0:10][0]

Prediction(uid=654, iid='Geoff Tate\\James Lomenzo\\Ray Luzier\\Carlos Cavazo', r_ui=0.001134, est=0.14290239137611155, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [90]:
user_predictions=list(filter(lambda x: x[0]==962,predictions))
user_predictions

[Prediction(uid=962, iid='Mark Snow', r_ui=0.02994, est=0.14290239137611155, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=962, iid='!!!', r_ui=0.149701, est=0.2346935855031467, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=962, iid='Duke Ellington & His Orchestra', r_ui=0.02994, est=0.15718944141103017, details={'actual_k': 41, 'was_impossible': False}),
 Prediction(uid=962, iid='Shelby Flint', r_ui=0.08982, est=0.14290239137611155, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=962, iid='William Orbit', r_ui=0.269461, est=0.08731485691909047, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=962, iid='Los Lobos', r_ui=0.11976, est=0.1057649786591608, details={'actual_k': 49, 'was_impossible': False}),
 Prediction(uid=962, iid='Saqi', r_ui=0.08982, est=0.14290239137611155, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(

In [9]:
pd_predictions=pd.DataFrame(predictions)
pd_predictions

,uid,iid,r_ui,est,details
0,545,Gone,1.250000,0.770561,"{'actual_k': 50, 'was_impossible': False}"
1,680,Sideways,0.021552,0.281562,"{'actual_k': 41, 'was_impossible': False}"
2,635,My Kinda Lover,0.263158,0.274795,"{'was_impossible': True, 'reason': 'Not enough..."
3,691,Ice Cold Man,0.054945,0.274795,"{'was_impossible': True, 'reason': 'Not enough..."
4,388,A New Day,0.106383,0.199145,"{'actual_k': 29, 'was_impossible': False}"
...,...,...,...,...,...
881581,182,Adaran,0.151515,0.274795,"{'was_impossible': True, 'reason': 'User and/o..."
881582,458,Monkey Man,0.054348,0.410502,"{'actual_k': 50, 'was_impossible': False}"
881583,751,Mad World,0.108911,0.648215,"{'actual_k': 50, 'was_impossible': False}"
881584,807,The Collector,0.096154,0.456296,"{'actual_k': 50, 'was_impossible': False}"


In [21]:
!pip install mysql
!pip install mysql.connector
!pip install mysql-connector-python-rf
import mysql.connector
from mysql.connector import errorcode
#Configuración de la conexión a Mysql
try:
  cnx = mysql.connector.connect(user='user_taller1', password='taller1.', host='127.0.0.1', database='taller1')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

cursor = cnx.cursor()

In [22]:
sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'coseno','user')"

In [13]:
pd_predictions['iid'][0][0:250]

'Gone'

In [23]:
def isNaN(string):
    return string != string
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions
for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
#del pd_predictions

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [46]:
ratings_muestra =  ratings[ :1000]
ratings_muestra

,user_id,artist_name,rating
0,1,[Unknown],0.005848
1,1,→Pia-No-Jac←,0.192982
2,1,□□□,0.046784
3,1,くるり,0.321637
4,1,はっぴいえんど,0.005848
...,...,...,...
995,2,Édith Piaf,0.026960
996,2,Editors,0.217752
997,2,Edwin Mccain,0.053920
998,2,Eels,0.161759


In [24]:
## coseno item-item
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=3, min_k=1, sim_options=sim_options)
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions


Computing the cosine similarity matrix...
Done computing similarity matrix.


,uid,iid,r_ui,est,details
0,1,Music For 18 Musicians: Section V,0.071429,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
1,2,Boy With A Coin,0.071942,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
2,4,Across 110Th Street,0.016234,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
3,2,Fuck Me Pumps,0.251799,0.851319,"{'actual_k': 3, 'was_impossible': False}"
4,1,Nude Rider,0.142857,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
...,...,...,...,...,...
3995,4,No Milk Today,0.032468,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
3996,2,Sick Sad Little World,0.107914,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."
3997,2,Dub Latina,0.179856,0.047961,"{'actual_k': 3, 'was_impossible': False}"
3998,2,T.H.T.,0.035971,0.274136,"{'was_impossible': True, 'reason': 'User and/o..."


In [25]:
sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'coseno','item')"
def isNaN(string):
    return string != string
for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
del pd_predictions

In [26]:
## Jaccard user
sim_options = {'name': 'pearson_baseline',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions
sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'Jaccard','user')"
def isNaN(string):
    return string != string

for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
del pd_predictions

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [27]:
## Jaccard item
sim_options = {'name': 'pearson_baseline',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions
sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'Jaccard','item')"
def isNaN(string):
    return string != string

for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
del pd_predictions

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [28]:
## pearson user
sim_options = {'name': 'pearson',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions

sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'Pearson','user')"
def isNaN(string):
    return string != string

for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
del pd_predictions

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [29]:
## pearson item
sim_options = {'name': 'pearson',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)
ratings_muestra =  ratings[ :20000]
surprise_dataset = Dataset.load_from_df( ratings_muestra[ ["user_id","track_name","rating"] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)
predictions = algo.fit(train_set).test(test_set)
pd_predictions=pd.DataFrame(predictions)
pd_predictions
sql = "INSERT INTO `recomendations_tracks` (`user_id`, `track_name`,`recomendation_score`,`model`,`tipo_modelo`) VALUES (%s, %s, %s,'Jaccard','item')"
def isNaN(string):
    return string != string

for i in pd_predictions.index: 
    cursor.execute(sql, (int(pd_predictions['uid'][i]),pd_predictions['iid'][i][0:250],float(pd_predictions['est'][i])))
    cnx.commit()
del pd_predictions

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [30]:
#print(df_artist)

cursor.close()
cnx.close()